- Author: Ben Du
- Date: 2021-04-19 20:58:22
- Title: Trait in Rust
- Slug: trait-in-rust
- Category: Computer Science
- Tags: Computer Science, programming, Rust, Copy, Clone, trait, super, generic
- Modified: 2021-06-18 09:41:45


**Things on this page are fragmentary and immature notes/thoughts of the author. Please read with your own judgement!**

## Trait 

Currently, 
a trait in Rust cannot access fields. 
If a default implementation is provided,
the implementation cannot refer to any struct field. 

[Best Practices When Defining a Default Implementation for a Trait’s Method](https://users.rust-lang.org/t/best-practices-when-defining-a-default-implementation-for-a-traits-method/2033/2)

[Allow fields in traits that map to lvalues in impl'ing type #1546](https://github.com/rust-lang/rfcs/pull/1546)

[Fields in Traits](https://internals.rust-lang.org/t/fields-in-traits/6933)

## PartialEq, Eq and Hash

1. Unlike other popular programming languages,
    Rust introduces the trait PartialEq 
    (for solving issues such as NaN in floating numbers).
    The `Eq` trait is a trait without any method 
    which indicating that an object of the struct implementing `Eq` is comparable to itself.
    You **always need to implement the trait `PartialEq` if you implement `Eq`**.
    This eventually means that if you implement any of `Eq` or `Hash`,
    you need to implement all three of `PartialEq`, `Eq` and `Hash`.
    One simple way to do this is to use the macro `#[derive(PartialEq, Eq, Hash)]`.
    However, 
    you can implement `PartialEq` without implementing `Eq` or `Hash`. 

## Sized vs ?Sized 

## Sync vs !Sync

## Copy vs Clone 

1. Clone means the type can be duplicated.
    Copy means the type can be duplicated by copying bytes.
    This means that Copy implies Clone,
    so when you implements Copy you should always implement Clone.

3. `Clone` is a common trait for the ability to explicitly duplicate an object.
    It differs from `Copy` in that `Copy` is implicit and extremely inexpensive, 
    while `Clone` is always explicit and may or may not be expensive. 
    In order to enforce these characteristics, 
    Rust does not allow you to reimplement `Copy`, 
    but you may reimplement `Clone` and run arbitrary code.
    Since `Clone` is more general than `Copy`, 
    you can automatically make anything `Copy` be `Clone` as well.
    

2. If a type does not implement the `Copy` trait,
    it is moved when passed as a parameter. 
    This might cause issues of "object moved".
    To resolve this issue, 
    you have to implement the `Copy` trait. 
    A simple way is to drive the `Copy` and `Clonable` traits
    using `#[derive(Copy, Clone)]`.


[Moves, copies and clones in Rust](https://hashrust.com/blog/moves-copies-and-clones-in-rust/)

[Trait std::clone::Clone](https://doc.rust-lang.org/std/clone/trait.Clone.html)

## Super/Sub Trait and Generic Functions

RFC: Supertrait item shadowing #2845

[https://github.com/rust-lang/rfcs/pull/2845]

In [14]:
trait Super {
    fn foo(&self);
}

trait Sub: Super {
    fn foo(&self);
}

impl Super for i32 {
    fn foo(&self) { 
        println!("super"); 
    }
}

impl Sub for i32 {
    fn foo(&self) { 
        println!("sub"); 
    }
}

fn super_generic_fn<S: Super>(x: S) {
    x.foo();
}

fn sub_super_generic_fn<S: Sub>(x: S) {
    generic_fn(x);
}

fn sub_generic_fn<S: Sub>(x: S) {
    x.foo();
}

Error: multiple applicable items in scope

In [8]:
let x: i32 = 42;
x.foo()

Error: candidate #1 is defined in an impl of the trait `Super` for the type `i32`

In [11]:
let x: i32 = 42;
sub_generic_fn(x);

super


## References 

[How can I implement PartialEq?](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html#how-can-i-implement-partialeq)

[How can I implement Eq?](https://doc.rust-lang.org/std/cmp/trait.Eq.html#how-can-i-implement-eq)

https://doc.rust-lang.org/std/cmp/trait.PartialEq.html